In [1]:
import cv2
import numpy as np
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from face_segmenter import segment_image

In [2]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]).astype('uint8')

In [3]:
csv_path = "/media/nacho/Disco Auxiliar/datasets/CelebA/list_bbox_celeba.csv"
images_dir = "/media/nacho/Disco Auxiliar/datasets/CelebA/img_align_celeba/img_align_celeba"

In [4]:
images_properties = pd.read_csv(csv_path,sep=",",nrows=100000)

In [11]:
x1 = 50
y1 = 70
w1 = 80
h1 = 100
faces_list = []
for _,image in images_properties.iterrows():
    image_path,bbox1,bbox2,bbox3,bbox4 = image.tolist()
    image_path = os.path.join(images_dir,image_path)
    img = cv2.imread(image_path)
    img = rgb2gray(img)
    face = img[y1:y1+h1,x1:x1+w1]
    face = cv2.resize(face,(32,32))
    faces_list.append(face)

In [12]:
tile_list = []
for face in faces_list:
    tiles,_ = segment_image(face,8)
    tile_list.append(tiles)

In [8]:
natural_images_path = "/media/nacho/Disco Auxiliar/datasets/natural_images"

natural_images_list = []

for data_folder in os.listdir(natural_images_path):
    if data_folder == "person": continue
    for image in os.listdir(os.path.join(natural_images_path,data_folder)):
        img_npy = cv2.imread(os.path.join(natural_images_path,data_folder,image))
        img_npy = rgb2gray(img_npy)
        subregiones,_ = segment_image(img_npy,8)
        natural_images_list.append(subregiones)

natural_images = np.concatenate(natural_images_list)

In [10]:
natural_images.shape

(5310640, 8, 8)

In [23]:
tiles_np = np.concatenate(tile_list)

In [14]:
tiles_np.shape

(1600000, 8, 8)

In [18]:
train_tiles_np = np.concatenate([tiles_np,natural_images])

In [19]:
pipe = Pipeline([('pca', PCA(0.99)), ('kmeans', KMeans(64))])

In [20]:
pipe.fit(train_tiles_np.reshape(-1,64))

/home/nacho/Documents/MIARFID/BIOM/SnK/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Pipeline(steps=[('pca', PCA(n_components=0.99)),
                ('kmeans', KMeans(n_clusters=64))])

In [21]:
from joblib import dump
dump(pipe,"pipe_pca099_kmeans64.joblib")

['pipe_pca099_kmeans64.joblib']

In [22]:
from joblib import load
pipe = load('pipe_pca099_kmeans64.joblib')

In [24]:
y_prime = pipe.predict(tiles_np.reshape(-1,64))

In [25]:
y_prime

array([46,  7, 48, ..., 22, 22, 37], dtype=int32)

In [26]:
y_prime_pos = y_prime.reshape(-1,16)

In [28]:
P_pos_q_cara_list = []
for i in range(y_prime_pos.shape[1]):
    P_pos_q_cara_i, _ = np.histogram(y_prime_pos[:,i],bins=range(0,65)) 
    P_pos_q_cara_list.append(P_pos_q_cara_i)

In [29]:
P_pos_q_cara = np.stack(P_pos_q_cara_list)

In [30]:
P_q_cara,_ = np.histogram(y_prime,bins=range(0,65))

In [31]:
P_q_cara

array([25340, 55417, 41005, 20588,  2200,  9413, 19442, 21914, 29618,
       40423, 28318, 18462, 37630,     0, 26029, 26190, 60723, 32211,
        8220, 16605, 20177,  5608, 39597, 41293, 16524,  7284, 30988,
       48849, 38558, 26355,  6587, 23131, 14246, 18987, 42281, 16172,
       30423, 34847, 27825,  7836,     0,     0, 51553, 23626, 16970,
       47362, 21772, 46772,  8225, 27540, 18988, 45245, 58203, 53354,
       12872, 22713,  4571, 12610, 18910, 29810, 18631,     0, 39655,
        3302])

In [32]:
np.save("P_q_cara.npy",P_q_cara)

In [33]:
np.save("P_pos_q_cara.npy",P_pos_q_cara)

In [7]:
natural_images_path = "/media/nacho/Disco Auxiliar/datasets/natural_images"

natural_images_list = []

for data_folder in os.listdir(natural_images_path):
    if data_folder == "person": continue
    for image in os.listdir(os.path.join(natural_images_path,data_folder)):
        img_npy = cv2.imread(os.path.join(natural_images_path,data_folder,image))
        img_npy = rgb2gray(img_npy)
        subregiones,_ = segment_image(img_npy,8)
        natural_images_list.append(subregiones)
    

In [34]:
natural_subregions = np.concatenate(natural_images_list,axis=0)

In [35]:
y_naturales_prime = pipe.predict(natural_subregions.reshape(-1,64))

In [36]:
P_q_nocara,_ = np.histogram(y_naturales_prime,bins=range(0,65))

In [37]:
P_q_nocara

array([264957, 184592, 229824, 294457, 160046,  14598,  61072, 158768,
        35462,  52969,  38486,  39621,  20729,      0,  50892,  31553,
       196016,   7716, 231692,  15676, 270070,  22736,  13346,  31949,
        33909,  22592, 237425, 215872,  14692,  36640,  24790,  17849,
        17662,  29316,   6211,  27092,  56374,  35552,  53699,  21410,
            0,      0,  51779,  15508, 288942, 222084,  20188,   8631,
       138353,  39102, 276779,  54914,  42339, 207739,  48866,  50953,
        24208,  22319,  54958, 255482,  16610,      0, 171253,  21321])

In [38]:
np.save("P_q_nocara.npy",P_q_nocara)

## Calcular el resultado

In [33]:
P_norm_q_nocara = P_q_nocara / P_q_nocara.sum()

In [34]:
P_norm_q_cara = P_q_cara / P_q_cara.sum()

In [35]:
P_pos_q_cara.sum(axis=0)

array([32094, 54222, 25041, 45680, 31461, 32514, 30685, 18467, 19298,
       16893, 24972, 37314, 14561, 16127, 13171, 46596, 17023, 15332,
       15051, 21556, 13576,  6964, 15547, 35692, 14952, 19123, 54018,
       11622, 15052,  7972, 16764, 37307, 34299, 28803, 52571, 12367,
       18361, 40813, 24377, 15683, 35383, 14224, 25451, 17477, 33925,
       47754, 14303, 14099, 16891, 30699, 10569, 30346, 18090, 48605,
       25200, 12136, 11798, 13123, 40996, 49845, 27492, 29572, 14042,
       14059])

In [36]:
P_norm_pos_q_cara = P_pos_q_cara / P_pos_q_cara.sum(axis=0)

In [37]:
P_norm_pos_q_cara.shape

(16, 64)

In [38]:
test = np.asarray([10,20,30,40,50,60])

In [39]:
test

array([10, 20, 30, 40, 50, 60])

In [41]:
P_norm_q_nocara[20],P_norm_q_nocara[30],P_norm_q_nocara[40]

(0.008417508417508417, 0.0016835016835016834, 0.005050505050505051)

In [42]:
den = np.prod(P_norm_q_nocara[test]/ (16*64)) 

In [43]:
np.prod(P_norm_pos_q_cara[np.arange(0,64,1,dtype=int),test])

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (64,) (6,) 

In [44]:
num = np.prod(P_norm_pos_q_cara[np.arange(0,6,1,dtype=int),test]) * np.prod(P_norm_q_cara[test])